<a href="https://colab.research.google.com/github/m-manuelmussa/Desenvolvimento-de-modelos-de-machine-learning-para-predizer-novos-inibidores-da-TK-HER2/blob/main/Desenvolvimento_de_modelos_de_machine_learning_baseados_em_QSAR_3D_para_a_predi%C3%A7%C3%A3o_de_novos_candidatos_a_f%C3%A1rmacos_TKI_HER2_para_o_tratamento_de_c%C3%A2ncer_da_mama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hello!
